In [1]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import json
import git


In [2]:
# get commit number for github repository
repo = git.Repo(search_parent_directories=True)  
sha = repo.head.object.hexsha
print(sha)

7402526bbadfcd422647cc60c007497dbe7af540


In [3]:
# setup basic arguments for qed-ci calculation
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}


cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}



In [4]:
# set up base dictionary - some of this will be updated with each calculation
dictionary = {
  "repo" : {
      
      "repository_url" : " https://github.com/FoleyLab/SCQED-PCQED",
      "branch" : "ruby_qed",
      "commit" : sha
  },
    
  "molecule" : {
    "molecule_name": "LiH",
    "geometry": [],
    "symbols": [
      "Li",
      "H"
    ],
    "z-matrix" : mol_str,
    "bond_length" : 1.8,
  },
    "driver": "energy",
    "model" : {
        "method" : "pCQED",
        "orbital_basis"  : options_dict["basis"],
        "photon_basis" : "number_basis",
        "number_of_photon_states" : 10,
        "number_of_electronic_states" : 25,
        
        "lambda" : [
            0,
            0,
            0,
        ],
        "omega" : 0,
    },
    
    "return_result" : [
        
    ],
    "cavity_free_rhf_energy" : 0.,
    "cavity_free_ci_ground_state_energy" : 0.,
  
"New Dict" : {  
"_________________________________" : "_________________________________",
"__________________________ ": "New dict for next r value",
"_________________________________" : "_________________________________",
  },   
}


# function to generate file names based on system details
def generate_file_name(dic):
#     string_r = "{:.3f}".format(r)
    file_name = dic["molecule"]["molecule_name"] + "_"
#     file_name += "r_" + string_r + "_"
    file_name += str(dictionary["model"]["method"]) + "_"
    file_name += str(dictionary["model"]["orbital_basis"]) + "_"
    file_name += str(dictionary["model"]["photon_basis"]) + "_"
    file_name += str(dictionary["model"]["number_of_photon_states"]) + "_"
    file_name += str(dictionary["model"]["number_of_electronic_states"]) + "_"
    file_name += "lambda_" + str(dictionary["model"]["lambda"][0]) 
    file_name += "_" + str(dictionary["model"]["lambda"][1]) 
    file_name += "_" + str(dictionary["model"]["lambda"][2]) + "_"
    file_name += "omega_" + str(dictionary["model"]["omega"]) + ".json"
    return file_name

In [5]:
print(dictionary)
print(generate_file_name(dictionary))

{'repo': {'repository_url': ' https://github.com/FoleyLab/SCQED-PCQED', 'branch': 'ruby_qed', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n', 'bond_length': 1.8}, 'driver': 'energy', 'model': {'method': 'pCQED', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'number_of_electronic_states': 25, 'lambda': [0, 0, 0], 'omega': 0}, 'return_result': [], 'cavity_free_rhf_energy': 0.0, 'cavity_free_ci_ground_state_energy': 0.0, 'New Dict': {'_________________________________': '_________________________________', '__________________________ ': 'New dict for next r value'}}
LiH_pCQED_sto-3g_number_basis_10_25_lambda_0_0_0_omega_0.json


In [6]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50
N_el = 2
N_ph = 10
omega = 0.12086
lambda_vector = np.array([0., 0., 0.05])

r_array = np.linspace(1.4, 2.2, N_R)

E_array = np.zeros((N_R, N_el))
list_of_dict = []
r_idx = 0
for r in r_array:
    file_name = generate_file_name(dictionary) 
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    dictionary["molecule"]["z-matrix"] = mol_str
    dictionary["molecule"]["bond_length"] = r
    
    
    #psi4.set_options({'restricted_docc': [3],'active': [4],'num_roots':2})
    scf_e, wfn = psi4.energy('SCF/sto-3g', return_wfn=True)
    #fci_energy = psi4.energy('fci',ref_wfn=wfn)    
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)
    #singlet_states = LiH.sort_dipole_allowed_states( 2)
    LiH.build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector)
    dictionary["model"]["number_of_photon_states"] = N_ph
    dictionary["model"]["number_of_electronic_states"] = N_el
    dictionary["model"]["lambda"] = [lambda_vector[0], lambda_vector[1], lambda_vector[2]]
    dictionary["model"]["omega"] = omega
    
    print(LiH.PCQED_H_PF)
    if (N_el * N_ph) <=4:
        dictionary["return_result"] = [LiH.PCQED_eigs[0], LiH.PCQED_eigs[1], LiH.PCQED_eigs[2], LiH.PCQED_eigs[3]]
        
    else:
        dictionary["return_result"] = [LiH.PCQED_eigs[0], LiH.PCQED_eigs[1], LiH.PCQED_eigs[2], LiH.PCQED_eigs[3], LiH.PCQED_eigs[4],LiH.PCQED_eigs[5]]
    dictionary["cavity_free_rhf_energy"] = scf_e
    dictionary["cavity_free_ci_ground_state_energy"] = LiH.CIeigs[0]
    geometry = np.asarray(mol.geometry())
    geometry_list = geometry.tolist()
    dictionary["molecule"]["geometry"] = geometry_list
    list_of_dict.append(dictionary)
    #file_name = generate_file_name(dictionary, r)
    #E_array[r_idx, :] = LiH.CIeigs[:50]
    #r_idx += 1
    #print(dictionary)
print('test',list_of_dict)
json_object = json.dumps(list_of_dict, indent=4)
with open(file_name, "w") as outfile:
    outfile.write(json_object)




Start SCF iterations:

Canonical RHF One-electron energy = -12.6946346236296037
CQED-RHF One-electron energy      = -12.6946346236296037
Nuclear repulsion energy          = 1.1339511657214287
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8605386610570473   dE = -7.86054E+00   dRMS = 1.78805E-13
SCF Iteration   2: Energy = -7.8605386610570385   dE =  8.88178E-15   dRMS = 3.97663E-14
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.86053866 hartree
Psi4  SCF energy: -7.86053866 hartree
 Completed QED-RHF in 0.08800911903381348 seconds
 Completed 1HSO Build in 5.1975250244140625e-05 seconds
 Completed ERI Build in 0.001149892807006836 seconds 
 Completed 2D build in 6.914138793945312e-05 seconds
 Completed 1G build in 1.3828277587890625e-05 seconds
 Completed the Dipole Matrix Build in 3.814697265625e-05 seconds
 Completed determinant list in 0.00045180320739746094 seconds 
 Completed constant offset matrix in 0.00013923645019531